# 8월 유저 데이터셋 전처리 과정 정리
# behaviors.tsv에서 impression log를 가지고 정리
# 뉴스 1개 해당 뉴스를 클릭한 사용자자 1명과 해당 뉴스를 과거, 미래에도 클릭하지 않을 사용자들 20명

In [12]:
# train userdataset 생성하기
import pandas as pd
import random
from tqdm import tqdm
from collections import defaultdict

random.seed(42)  # 재현성

# =========================================================
# 1) 데이터 불러오기 (헤더 주의)
# =========================================================
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
train_behaviors_df = pd.read_csv(
    'download/MINDsmall_train/behaviors.tsv',
    sep='\t',
    names=BEHAVIOR_COLUMNS,
    header=None,         # <<< 헤더 없음!
    dtype=str
)

news_df = pd.read_csv(
    "news_times_global.csv",
    parse_dates=['publish_time','lifespan_end']
)

# news_id 컬럼명 정규화
if 'NewsID' in news_df.columns:
    news_id_col = 'NewsID'
elif 'news_id' in news_df.columns:
    news_id_col = 'news_id'
else:
    raise ValueError("news_times_global.csv에 NewsID/news_id 컬럼이 필요합니다.")

allowed_news_ids = set(news_df[news_id_col].astype(str))

# =========================================================
# 2) train impression에서 클릭 뉴스 개수 확인 (옵션)
# =========================================================
click_news_count = 0
for impression_row in train_behaviors_df["Impressions"]:
    if pd.isna(impression_row):
        continue
    for impression in str(impression_row).split():
        nid, label = impression.split("-")
        if label == '1':
            click_news_count += 1
print('train impressions-1인 뉴스 개수: ', click_news_count)

# =========================================================
# 3) 유저별 클릭 리스트 만들기 (label==1만)
# =========================================================
train_user_list = []
for _, row in tqdm(train_behaviors_df.iterrows(), total=len(train_behaviors_df), desc="Build user click list"):
    user = str(row['UserID'])
    time = str(row['Time'])
    imp_str = str(row['Impressions'])
    imp_list = []
    if imp_str and imp_str != 'nan':
        for imp in imp_str.split():
            news_id, label = imp.split('-')
            if label == '1':
                imp_list.append(imp)  # "newsid-1" 형태 저장
    train_user_list.append({
        'UserID': user,
        'Time': time,
        'ClickNews': imp_list
    })

# 전체 유저 집합
all_user_ids = set(u['UserID'] for u in train_user_list)

# =========================================================
# 4) 뉴스별 -> (클릭한 유저, 클릭 시각) 리스트
# =========================================================
news_click_users = defaultdict(list)  # news_id -> [(uid, time), ...]
for user in train_user_list:
    uid = user['UserID']
    time = user['Time']
    for imp in user['ClickNews']:
        if '-' in imp:
            news_id, label = imp.split('-')
            if label == '1':
                news_click_users[news_id].append((uid, time))

# =========================================================
# 5) 사용자-데이터셋 생성
#    조건:
#    - news_id는 allowed_news_ids에 속하는 것만
#    - 각 행은 [클릭유저 1명 + 미클릭 유저 20명]
#    - 한 뉴스에 클릭 유저가 여러 명이면 여러 행 생성
# =========================================================
train_overlap_news_list = []
impression_id = 1

for news_id in tqdm(allowed_news_ids, desc="Generate user-dataset rows (train)"):
    clicked_list = news_click_users.get(news_id, [])
    if not clicked_list:
        continue  # 해당 뉴스의 클릭이 없다면 행 생성 X

    clicked_user_ids = set(uid for uid, _ in clicked_list)
    other_user_ids = list(all_user_ids - clicked_user_ids)

    # 항상 20명 보장 (부족하면 에러로 알림)
    if len(other_user_ids) < 20:
        raise ValueError(f"[{news_id}] 비클릭 유저가 20명 미만입니다. (현재 {len(other_user_ids)}명)")

    for uid, clicked_time in clicked_list:
        # 20명 정확히 뽑기
        sampled_non_click_ids = random.sample(other_user_ids, 20)

        impression_users = [f"{uid}-1"] + [f"{nid}-0" for nid in sampled_non_click_ids]
        random.shuffle(impression_users)

        train_overlap_news_list.append({
            'ImpressionID': impression_id,
            'NewsID': news_id,
            'Time': clicked_time,
            'ImpressionUsers': ' '.join(impression_users)
        })
        impression_id += 1

# =========================================================
# 6) 저장
# =========================================================
out_df = pd.DataFrame(train_overlap_news_list, columns=['ImpressionID','NewsID','Time','ImpressionUsers'])
out_df.to_csv('train_user_dataset.tsv', sep='\t', index=False, header=False)

print(f"생성 행 수: {len(out_df)}")
# sanity print
print(out_df.head(3))


train impressions-1인 뉴스 개수:  236344


Generate user-dataset rows (train): 100%|██████████| 9100/9100 [00:15<00:00, 579.90it/s]


생성 행 수: 236344
   ImpressionID  NewsID                   Time  \
0             1  N49465  11/13/2019 5:57:11 AM   
1             2   N3367  11/11/2019 6:29:06 AM   
2             3   N9657  11/12/2019 8:27:57 AM   

                                     ImpressionUsers  
0  U66111-0 U90945-0 U32004-0 U46065-0 U64913-0 U...  
1  U66980-0 U81244-0 U113-0 U67679-1 U48335-0 U65...  
2  U79165-0 U14850-1 U2487-0 U57220-0 U58850-0 U4...  


In [13]:
# train user history 파일 생성

user_history = {}

for _, row in train_behaviors_df.iterrows():
    user = row['UserID']
    history = row['History']
    user_history[user] = history

history_df = pd.DataFrame(list(user_history.items()), columns=["UserID", "History"])

history_df.to_csv("train_user_history.tsv", sep="\t", index=False, header=False)

In [14]:
# dev behaviors 불러오기

import pandas as pd
import random
from tqdm import tqdm
from collections import defaultdict

# =========================
# 설정
# =========================
NEG_PER_IMP = 20
RNG_SEED = 42
random.seed(RNG_SEED)

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# 1) dev behaviors.tsv 읽기 (헤더 없음!)
dev_behaviors_df = pd.read_csv(
    'download/MINDsmall_dev/behaviors.tsv',
    sep='\t',
    names=BEHAVIOR_COLUMNS,
    header=None,        # <<< 중요: 헤더 없음
    dtype=str
)

# 2) 허용 뉴스 집합 준비 (newsID 컬럼명 안전화)
news_df = pd.read_csv("news_times_global.csv", parse_dates=['publish_time','lifespan_end'])
if 'NewsID' in news_df.columns:
    news_id_col = 'NewsID'
elif 'news_id' in news_df.columns:
    news_id_col = 'news_id'
else:
    raise ValueError("news_times_global.csv에 NewsID/news_id 컬럼이 필요합니다.")
allowed_news_ids = set(news_df[news_id_col].astype(str))

# 3) dev impression에서 클릭 뉴스 개수 확인 (옵션)
click_news_count = 0
for impression_row in dev_behaviors_df["Impressions"]:
    if pd.isna(impression_row):
        continue
    for impression in str(impression_row).split():
        nid, label = impression.split("-")
        if label == '1':
            click_news_count += 1
print('dev impressions-1인 뉴스 개수:', click_news_count)

# 4) 유저별 클릭 리스트 만들기 (label==1만)
dev_user_list = []
for _, row in tqdm(dev_behaviors_df.iterrows(), total=len(dev_behaviors_df), desc="Build dev user click list"):
    user = str(row['UserID'])
    time = str(row['Time'])
    imp_str = str(row['Impressions'])
    imp_list = []
    if imp_str and imp_str != 'nan':
        for imp in imp_str.split():
            news_id, label = imp.split('-')
            if label == '1':
                imp_list.append(imp)  # "newsid-1"
    dev_user_list.append({
        'UserID': user,
        'Time': time,
        'ClickNews': imp_list
    })

# 5) 전체 유저 집합
all_user_ids = set(u['UserID'] for u in dev_user_list)

# 6) 뉴스별 -> (클릭한 유저, 클릭시각) 리스트 (중복 유지)
news_click_users = defaultdict(list)  # news_id -> [(uid, time), ...]
for user in dev_user_list:
    uid = user['UserID']
    time = user['Time']
    for imp in user['ClickNews']:
        if '-' in imp:
            news_id, label = imp.split('-')
            if label == '1':
                news_click_users[news_id].append((uid, time))

# 7) 사용자-데이터셋 생성
#    - news_id는 allowed_news_ids만 대상
#    - 각 클릭 유저별로 한 행 생성(중복 유지)
#    - 네거티브는 "그 뉴스 클릭 유저 전체"를 제외한 유저에서 정확히 20명
dev_overlap_news_list = []
impression_id = 1

for news_id in tqdm(allowed_news_ids, desc="Generate user-dataset rows (dev)"):
    clicked_list = news_click_users.get(news_id, [])
    if not clicked_list:
        continue  # dev에서 실제 클릭 없는 뉴스는 스킵

    # 이 뉴스의 "모든" 클릭 유저 집합
    all_clicked_uids_for_news = set(uid for uid, _ in clicked_list)
    neg_pool = list(all_user_ids - all_clicked_uids_for_news)

    # 항상 20 보장 (부족하면 에러)
    if len(neg_pool) < NEG_PER_IMP:
        raise ValueError(f"[{news_id}] dev에서 비클릭 유저가 {NEG_PER_IMP}명 미만입니다. (현재 {len(neg_pool)}명)")

    for uid, clicked_time in clicked_list:
        pos = f"{uid}-1"
        sampled_neg_ids = random.sample(neg_pool, NEG_PER_IMP)
        negs = [f"{nid}-0" for nid in sampled_neg_ids]

        impression_users = [pos] + negs
        random.shuffle(impression_users)

        dev_overlap_news_list.append({
            'ImpressionID': impression_id,
            'NewsID': news_id,
            'Time': clicked_time,
            'ImpressionUsers': ' '.join(impression_users)
        })
        impression_id += 1

# 8) 저장
out_cols = ['ImpressionID','NewsID','Time','ImpressionUsers']
dev_out_df = pd.DataFrame(dev_overlap_news_list, columns=out_cols)
dev_out_df.to_csv('dev_user_dataset.tsv', sep='\t', index=False, header=False)

print(f"dev 생성 행 수: {len(dev_out_df)}")
print(dev_out_df.head(3))


dev impressions-1인 뉴스 개수: 111383


Generate user-dataset rows (dev): 100%|██████████| 9100/9100 [00:04<00:00, 1827.78it/s]


dev 생성 행 수: 111383
   ImpressionID  NewsID                    Time  \
0             1  N52322   11/15/2019 3:45:34 PM   
1             2  N24713  11/15/2019 12:41:28 PM   
2             3  N24713   11/15/2019 1:51:03 PM   

                                     ImpressionUsers  
0  U64199-0 U88480-0 U62928-0 U82069-0 U82913-0 U...  
1  U17639-0 U58541-0 U79149-0 U79782-1 U56928-0 U...  
2  U73880-0 U6825-1 U65833-0 U40269-0 U88292-0 U5...  


In [15]:
# dev user history 파일 생성

user_history = {}

for _, row in dev_behaviors_df.iterrows():
    user = row['UserID']
    history = row['History']
    user_history[user] = history

history_df = pd.DataFrame(list(user_history.items()), columns=["UserID", "History"])

history_df.to_csv("dev_user_history.tsv", sep="\t", index=False, header=False)


In [17]:
import pandas as pd
from collections import defaultdict

# =========================
# 설정
# =========================
TRAIN_BEH = 'download/MINDsmall_train/behaviors.tsv'
DEV_BEH   = 'download/MINDsmall_dev/behaviors.tsv'
TRAIN_USR = 'train_user_dataset.tsv'
DEV_USR   = 'dev_user_dataset.tsv'
NEWS_TIME = 'news_times_global.csv'

NEG_PER_IMP = 20
STRICT_GLOBAL = True       # (B) 엄격 검증: train+dev 합집합 기준
CHECK_COVERAGE = True      # (C-1) 뉴스별 기대 행수(고유 클릭 유저 수) 커버리지 점검
CHECK_INTERNAL = True      # (C-2) 행 내부 충돌/중복 점검

# =========================
# 유틸 1: behaviors에서 뉴스별 '고유' 클릭 유저 집합 맵
# =========================
def build_clicked_map(behaviors_path: str):
    cols = ['ImpressionID','UserID','Time','History','Impressions']
    df = pd.read_csv(behaviors_path, sep='\t', names=cols, header=None, dtype=str)
    news_click_users = defaultdict(set)  # news_id -> {uid, ...}
    for imp_str, uid in zip(df['Impressions'], df['UserID']):
        if pd.isna(imp_str):
            continue
        for token in str(imp_str).split():
            nid, label = token.split('-')
            if label == '1':
                news_click_users[str(nid)].add(str(uid))
    return news_click_users

# =========================
# 유틸 2: behaviors에서 (a) 고유 클릭 유저 집합, (b) 총 클릭 수(중복 포함)
# =========================
def build_clicked_multimap(behaviors_path: str):
    cols = ['ImpressionID','UserID','Time','History','Impressions']
    df = pd.read_csv(behaviors_path, sep='\t', names=cols, header=None, dtype=str)
    nid_to_uids = defaultdict(set)     # news_id -> {uid,...} (고유)
    nid_to_clicks_total = defaultdict(int)  # news_id -> 총 클릭 수(중복 포함)
    for imp_str, uid in zip(df['Impressions'], df['UserID']):
        if pd.isna(imp_str):
            continue
        for token in str(imp_str).split():
            nid, label = token.split('-')
            if label == '1':
                nid_to_uids[str(nid)].add(str(uid))
                nid_to_clicks_total[str(nid)] += 1
    return nid_to_uids, nid_to_clicks_total

# =========================
# 허용 뉴스 집합 로드 (컬럼명 안전화)
# =========================
news_df = pd.read_csv(NEWS_TIME, parse_dates=['publish_time','lifespan_end'])
if 'NewsID' in news_df.columns:
    news_id_col = 'NewsID'
elif 'news_id' in news_df.columns:
    news_id_col = 'news_id'
else:
    raise ValueError("news_times_global.csv에 NewsID/news_id 컬럼이 필요합니다.")
ALLOWED_NEWS = set(news_df[news_id_col].astype(str))

# =========================
# 클릭 맵 빌드 (train/dev/합집합)
# =========================
train_clicked = build_clicked_map(TRAIN_BEH)  # nid -> set(uids) (고유)
dev_clicked   = build_clicked_map(DEV_BEH)

# 합집합 맵 (엄격 검증용)
union_clicked = defaultdict(set)
for nid, s in train_clicked.items():
    union_clicked[nid] |= s
for nid, s in dev_clicked.items():
    union_clicked[nid] |= s

# (C-1) 커버리지 검증에 쓸 멀티맵(고유/총합)
train_clicked_unique, train_clicks_total = build_clicked_multimap(TRAIN_BEH)
dev_clicked_unique,   dev_clicks_total   = build_clicked_multimap(DEV_BEH)

# =========================
# 검증 A/B: 파일 단위 기본/엄격 검증
# =========================
def validate_user_dataset(tsv_path: str, split_name: str, clicked_map: dict, allowed_news: set):
    df = pd.read_csv(tsv_path, sep='\t',
                     names=['ImpressionID','NewsID','Time','ImpressionUsers'],
                     header=None, dtype=str)

    # (1) 허용 뉴스만 포함?
    bad_news = set(df['NewsID'].astype(str)) - allowed_news
    assert not bad_news, f"[{split_name}] 허용되지 않은 뉴스 포함: {list(bad_news)[:5]}"

    # (2) 각 행 구조/음성 조건 점검
    def _check_row(row):
        parts = str(row['ImpressionUsers']).split()
        ones  = [p for p in parts if p.endswith('-1')]
        zeros = [p for p in parts if p.endswith('-0')]

        assert len(ones) == 1,  f"[{split_name}] row {row['ImpressionID']}: 양성 수 != 1 (현재 {len(ones)})"
        assert len(zeros) == NEG_PER_IMP, f"[{split_name}] row {row['ImpressionID']}: 음성 수 != {NEG_PER_IMP} (현재 {len(zeros)})"

        nid = str(row['NewsID'])
        pos_uid = ones[0].rsplit('-', 1)[0]
        neg_uids = [z.rsplit('-', 1)[0] for z in zeros]

        # 양성 유저는 실제 클릭 유저
        assert pos_uid in clicked_map.get(nid, set()), \
            f"[{split_name}] row {row['ImpressionID']}: 양성 유저가 실제 클릭 유저가 아님"

        # 음성 유저는 해당 맵에서 클릭 유저에 포함되면 안 됨
        bad_negs = [u for u in neg_uids if u in clicked_map.get(nid, set())]
        assert not bad_negs, \
            f"[{split_name}] row {row['ImpressionID']}: 음성에 클릭 유저 섞임 => {bad_negs[:5]}"

    df.apply(_check_row, axis=1)
    print(f"✅ [{split_name}] 기본/엄격 검증 통과: 허용 뉴스 + (1양성,{NEG_PER_IMP}음성) + 음성=해당 기준 미클릭")

# =========================
# (A) 기본 검증: split 내부 기준
# =========================
validate_user_dataset(TRAIN_USR, "train(내부기준)", train_clicked, ALLOWED_NEWS)
validate_user_dataset(DEV_USR,   "dev(내부기준)",   dev_clicked,   ALLOWED_NEWS)


# =========================
# (B-1) 추가: (3) 행 커버리지(고유 클릭 유저 수 == 생성 행 수) 점검
# =========================
def check_split_coverage(tsv_path: str, split_name: str, clicked_unique_map: dict):
    df = pd.read_csv(tsv_path, sep='\t',
                     names=['ImpressionID','NewsID','Time','ImpressionUsers'],
                     header=None, dtype=str)

    # 뉴스별 생성 행 수
    rows_per_news = df.groupby('NewsID', as_index=False)['ImpressionID'].count()
    rows_map = dict(zip(rows_per_news['NewsID'].astype(str), rows_per_news['ImpressionID']))

    missing = []
    mismatch = []
    for nid, uids in clicked_unique_map.items():
        expected = len(uids)  # 고유 클릭 유저 수 = 기대 행 수 (설계: 클릭 유저 1명당 1행)
        if expected == 0:
            continue
        if (nid in ALLOWED_NEWS) and (nid not in rows_map):
            missing.append((nid, expected))
        elif (nid in ALLOWED_NEWS) and (rows_map[nid] != expected):
            mismatch.append((nid, expected, rows_map[nid]))

    if missing:
        print(f"[{split_name}] 행 미생성 뉴스(허용셋 & 클릭유저 존재): 예) {missing[:5]} ... 총 {len(missing)}개")
    if mismatch:
        print(f"[{split_name}] 기대 행수 불일치(기대!=실제): 예) {mismatch[:5]} ... 총 {len(mismatch)}개")
    if not missing and not mismatch:
        print(f"✅ [{split_name}] (3) 검증 통과: 고유 클릭 유저 수 만큼 행이 생성됨")

if CHECK_COVERAGE:
    check_split_coverage(TRAIN_USR, "train 커버리지", train_clicked_unique)
    check_split_coverage(DEV_USR,   "dev 커버리지",   dev_clicked_unique)

# =========================
# (B-2) 추가: 행 내부 충돌/중복 점검
# =========================
def check_row_internal_conflicts(tsv_path: str, split_name: str):
    df = pd.read_csv(tsv_path, sep='\t',
                     names=['ImpressionID','NewsID','Time','ImpressionUsers'],
                     header=None, dtype=str)

    bad_conflicts = []  # 양성/음성 동시 등장
    bad_dup_negs = []   # 음성 중복
    for _, row in df.iterrows():
        parts = str(row['ImpressionUsers']).split()
        pos = [p for p in parts if p.endswith('-1')]
        neg = [p for p in parts if p.endswith('-0')]
        pos_uids = [p.rsplit('-', 1)[0] for p in pos]
        neg_uids = [p.rsplit('-', 1)[0] for p in neg]

        # 1) 양성=음성 충돌
        inter = set(pos_uids) & set(neg_uids)
        if inter:
            bad_conflicts.append((row['ImpressionID'], list(inter)[:3]))

        # 2) 음성 중복
        if len(neg_uids) != len(set(neg_uids)):
            bad_dup_negs.append(row['ImpressionID'])

    if bad_conflicts:
        print(f"[{split_name}] ⚠️ 양성/음성 사용자 충돌: 예) {bad_conflicts[:5]} ... 총 {len(bad_conflicts)}행")
    if bad_dup_negs:
        print(f"[{split_name}] ⚠️ 음성 사용자 중복: 예) {bad_dup_negs[:5]} ... 총 {len(bad_dup_negs)}행")
    if not bad_conflicts and not bad_dup_negs:
        print(f"✅ [{split_name}] 행 내부 충돌/중복 없음")

if CHECK_INTERNAL:
    check_row_internal_conflicts(TRAIN_USR, "train 내부무결성")
    check_row_internal_conflicts(DEV_USR,   "dev 내부무결성")


✅ [train(내부기준)] 기본/엄격 검증 통과: 허용 뉴스 + (1양성,20음성) + 음성=해당 기준 미클릭
✅ [dev(내부기준)] 기본/엄격 검증 통과: 허용 뉴스 + (1양성,20음성) + 음성=해당 기준 미클릭
[train 커버리지] 기대 행수 불일치(기대!=실제): 예) [('N55689', 4257, 4316), ('N49685', 2273, 2294), ('N33619', 3214, 3246), ('N55204', 475, 481), ('N33885', 1040, 1043)] ... 총 558개
[dev 커버리지] 기대 행수 불일치(기대!=실제): 예) [('N31958', 7993, 8042), ('N23513', 2853, 2900), ('N5940', 4178, 4191), ('N15347', 501, 505), ('N24802', 2232, 2247)] ... 총 131개
✅ [train 내부무결성] 행 내부 충돌/중복 없음
✅ [dev 내부무결성] 행 내부 충돌/중복 없음
